In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
songs_base_df = pd.read_csv("./Resources/Spotify_Youtube.csv")
songs_base_df.head()

In [ ]:
# songs_base_df.columns

In [ ]:
songs_cleaned_df = songs_base_df.drop(columns='Unnamed: 0')
print(songs_cleaned_df.columns)
songs_cleaned_df.head()

In [ ]:
print(songs_cleaned_df.dtypes)

In [ ]:
#This is removing any object column that dosn't help with plots
songs_reduced_df = songs_base_df.drop(columns=['Unnamed: 0', 'Url_spotify', 'Album_type', 'Uri', 'Url_youtube', 
                                     'Title', 'Channel', 'Description', 'Licensed', 'official_video'])
print(songs_reduced_df.dtypes)
songs_reduced_df

In [ ]:
songs_reduced_df.isna().any()

In [ ]:
#Looking for NaNs
songs_reduced_df.count()

In [ ]:
##Cleaned all columns with NaN 
##This is our Baseline DataFrame --no long a top 10 per artest dataframe
songs_reduced_noNaN = songs_reduced_df.dropna(how='any') #['Likes']
songs_reduced_noNaN.count()

In [ ]:
top_100_liked_df = songs_reduced_noNaN.sort_values('Likes',ascending=False)
top_100_liked_df = top_100_liked_df.head(100)
top_100_liked_df

In [ ]:
top_100_liked_df.value_counts(subset='Artist')

In [ ]:
##not all are devided by 10 with using the _noNaN dataframe but has correct ratio
mean_artist_likes = songs_reduced_noNaN.groupby(["Artist"])["Likes"].agg('mean')
# mean_artist_likes

In [ ]:
artist_likes_summary = pd.DataFrame(
                        {"Average Likes": mean_artist_likes,
                         })
artist_likes_summary.sort_values('Average Likes', ascending=False)

In [ ]:
##show how many of the rows have at least one Null 
songs_reduced_df[songs_reduced_df.isna().any(axis=1)]  

In [ ]:
#This is pulled from the top 100 tracks some artist had more than one track on the top 100 likes
#This is a summary table
per_artist_summary = top_100_liked_df.groupby(['Artist']).agg({'Likes':'mean','Valence':'mean','Speechiness':'mean',
                                                           'Acousticness':'mean','Instrumentalness':'mean','Liveness':'mean',
                                                           'Energy':'mean'})
per_artist_summary_df = per_artist_summary.sort_values('Likes',ascending=False)
per_artist_summary_df = per_artist_summary_df.reset_index()
per_artist_summary_df

In [ ]:
#A df of the top liked tracks per artist no duplicate no dups
top_artist_track = songs_reduced_noNaN.sort_values('Likes',ascending=False).drop_duplicates(subset=['Artist'],keep='first')
top_artist_track = top_artist_track.drop_duplicates(subset=['Track'],keep='first')
top_artist_track.reset_index(inplace=True)
top_artist_track

In [ ]:
top_artist_track.to_csv("Resources/ArtistTopTrack.csv", index=False, header=True)

In [ ]:
#####################################################################################################
#####################################################################################################

In [ ]:
artist_toptrack_df = pd.read_csv("./Resources/ArtistTopTrack.csv")
artist_toptrack_df = artist_toptrack_df.drop(columns='index')
artist_toptrack_df.head()

In [ ]:
artist_and_likes = artist_toptrack_df[["Artist","Likes"]]
artist_and_likes = artist_and_likes.set_index("Artist")
#print(artist_and_likes.head(50))
artist_and_likes_top100 = artist_and_likes.head(50)
#print(artist_and_likes.tail(50))
artist_and_likes_bottom100 = artist_and_likes.tail(50)

In [ ]:
artist_and_likes_top100.plot(kind="bar", figsize=(10,5))
plt.title("Top 50 Artist by Likes")
plt.ylabel('Likes (Million)')
plt.tight_layout()
plt.savefig('./Images/Artist-Line-Top-50.png')
plt.show()

In [ ]:
artist_and_likes_bottom100.plot(kind="bar", figsize=(10,5))
plt.title("Bottom 50 Artist by Likes")
plt.ylabel('Likes')
plt.tight_layout()
plt.savefig('./Images/Artist-Line-Bottom-50.png')
plt.show()

In [ ]:
# artist_and_likes.plot(kind="bar")  #figsize=(20,3.5))  getting error for Joey Bada$$ 
# plt.show()

In [ ]:
ax = artist_and_likes_top100.plot(kind="barh", figsize=(5,10), legend=False)
plt.title("Top 50 Artist by Likes")
plt.xlabel('Likes (Millions)')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('./Images/Artist-Line-Top-50-vert.png')
plt.show()

In [ ]:
ax = artist_and_likes_bottom100.plot(kind="barh", figsize=(5,10), legend=False)
plt.title("Top 50 Artist by Likes")
plt.xlabel('Likes')
ax.invert_yaxis()
plt.tight_layout()
plt.savefig('./Images/Artist-Line-Bottom-50-vert.png')
plt.show()

In [ ]:
artist_and_likes.reset_index(inplace=True)

In [ ]:
artist_and_likes['Artist'] = artist_and_likes['Artist'].str.replace('$', '\$')

In [ ]:
# artist_and_likes = artist_and_likes.set_index("Artist")
# artist_and_likes.plot(kind="bar") #figsize=(10,3.5))

In [ ]:
ax = per_artist_summary_df.plot.scatter(y='Likes', x='Speechiness', color='green')
# Add the rest of the attributes to the scatter plot to the first one
per_artist_summary_df.plot.scatter(y='Likes', x='Liveness', color='blue', ax=ax)
per_artist_summary_df.plot.scatter(y='Likes', x='Valence', color='red', ax=ax)
per_artist_summary_df.plot.scatter(y='Likes', x='Energy', color='orange', ax=ax)
per_artist_summary_df.plot.scatter(y='Likes', x='Acousticness', color='yellow', ax=ax)
per_artist_summary_df.plot.scatter(y='Likes', x='Instrumentalness', color='violet', ax=ax)
# Set the x and y -axis label
ax.set_xlabel('Track Attributes')
ax.set_ylabel('Popularity')
# Add a legend in the upper left quadrant
ax.legend(['Valence', 'Liveness', 'Speechiness', 'Energy', 'Acousticness', 'Instrumentalness'], 
          loc='upper left', bbox_to_anchor=(.35, 1))
plt.savefig('./Images/artist_summary_plot.png')
plt.show()
# Calculate the correlation between each attribute and popularity
correlations = per_artist_summary_df.corr()['Likes']
print(correlations)

In [ ]:
ax = per_artist_summary_df.plot.scatter(y='Likes', x='Liveness')
plt.savefig('./Images/artist_liveness_plot.png')
plt.show()

In [ ]:
per_artist_summary_df.plot.scatter(y='Likes', x='Valence')
plt.savefig('./Images/artist_Valence_plot.png')
plt.show()

In [ ]:
per_artist_summary_df.plot.scatter(y='Likes', x='Speechiness')
plt.savefig('./Images/artist_speachiness_plot.png')
plt.show()